In [9]:
import pandas as pd
import iso8601
import pytz
from datetime import datetime
from time import sleep

from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest, GetOrdersRequest, GetOrderByIdRequest
from alpaca.trading.enums import OrderSide, OrderStatus, TimeInForce

from bars import Bars

apikey = "PKKJZYB9P6Q36H84YMXF"
secretkey = "hBnEAEiD1f67p6hM4DKkUBtixY01YulNWSuGHOyx"

trading_client = TradingClient(apikey, secretkey, paper=True)


def get_clock():
    global trading_client
    clock = trading_client.get("/clock")
    return clock


def get_datetime_aware() -> datetime:
    clock = get_clock()
    timestamp = clock["timestamp"]
    dt = iso8601.parse_date(timestamp)
    return dt


def get_datetime_tz(timezone) -> datetime:
    dt = get_datetime_aware()
    return dt.astimezone(timezone)


def get_datetime_eastern() -> datetime:
    return get_datetime_tz(pytz.timezone("US/Eastern"))


def get_datetime_utc() -> datetime:
    return get_datetime_tz(pytz.timezone("UTC"))

def wait_for_market_open():
    while True:
        c = get_clock()
        if c["is_open"]:
            print("Market Open")
            return True
        print("is_open", c["is_open"])
        time_now = iso8601.parse_date(c["timestamp"])
        print("time_now", time_now)
        next_open = iso8601.parse_date(c["next_open"])
        print("next_open", next_open)
        seconds_to_open = (next_open - time_now).seconds
        print("seconds_to_open", seconds_to_open)
        if seconds_to_open < 3600:
            print("waiting for", seconds_to_open, "seconds")
            sleep(seconds_to_open)
        else:
            print(seconds_to_open, "seconds is too long to wait for open")
            return False


In [5]:
get_clock()

{'timestamp': '2022-09-26T21:20:16.853632689-04:00',
 'is_open': False,
 'next_open': '2022-09-27T09:30:00-04:00',
 'next_close': '2022-09-27T16:00:00-04:00'}

In [7]:
wait_for_market_open()

is_open False
time_now 2022-09-26 21:32:26.034552-04:00
next_open 2022-09-27 09:30:00-04:00
seconds_to_open 43053
43053 seconds is too long to wait for open


False

In [10]:
wait_for_market_open()
bars = Bars()

is_open False
time_now 2022-09-26 22:02:52.865375-04:00
next_open 2022-09-27 09:30:00-04:00
seconds_to_open 41227
41227 seconds is too long to wait for open
